Make sure you understand that when you use opfunction rank in AlphaEngineer it is the rank of the inputted assets in the universe. Since we defined our topcap30 universe, we cannot use that to compute our alpha factors because there will be gaps in each stock's time series (price is nan for example ฟันหลอ). Instead we have to use the original ohlcv dataset we have because that data is complete. One problem is the opfunction rank will not compute over topcap30 universe but over the whole stock universe.

In [1]:
import pandas as pd
import nasdaqdatalink as ndl
import json
import numpy as np
from typing import List, Callable, Dict
from realgam.quantlib import general_utils as gu
from realgam.quantlib.engineer.op_engineer_vect import OpEngineerV
from realgam.quantlib.engineer.ta_engineer_vect import TalibEngineerV
from realgam.quantlib.engineer.alpha_engineer_vect import AlphaEngineerV
from realgam.quantlib.engineer.alpha_engineer import AlphaEngineer

from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

from talib import WMA
import talib
from joblib import Parallel, delayed
import datetime
import os
import time
PROJECT_PATH = os.getenv('QuantSystemMVP')
DATA_PATH = f'{PROJECT_PATH}/Data/historical/stock_hist_perma.obj'

In [2]:
# uni_path = os.path.join(PROJECT_PATH, 'Data/Projects/Alpha_ML/universe_top30.obj')
# uni_df = gu.load_file(uni_path)

In [2]:
stocks_df, stocks_extended_df, available_tickers = gu.load_file(DATA_PATH)
stacked_hist = stocks_df.copy()

In [3]:
openg = OpEngineerV(stacked_hist, 'permaticker', 'date')
openg.ts_ret(inplace=True)
uni_df = openg.df.copy()
uni_df.rename(columns={'ts_ret_closeadj': 'returns'}, inplace=True)

In [4]:
candle_name_dict = {'o': 'openadj', 'h': 'highadj', 'l': 'lowadj',
                    'c': 'closeadj', 'v': 'volume', 'r': 'returns'}

In [5]:
ae = AlphaEngineer(uni_df, 'permaticker', 'date', candle_name_dict)

In [6]:
def run_eng(alpha_engineer, method_name):

    return getattr(alpha_engineer, method_name)()

alpha_eng_index = [i for i in range(1,31,1)]
# alpha_eng_index = [1,2]
alpha_eng_names = [f'alpha{index}' for index in alpha_eng_index]

# logger.info('Begin')
s_time_chunk = time.time()
eng_values = Parallel(n_jobs=-1)(delayed(run_eng)(ae, eng_name) for eng_name in alpha_eng_names)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 1290.7024109363556 sec


In [7]:
for eng_name, eng_value in zip(alpha_eng_names, eng_values):
    uni_df[eng_name] = eng_value


In [8]:
SAVE_PATH = f'{PROJECT_PATH}/Data/Projects/Alpha_ML/alpha_eng_30.obj'
gu.save_file(SAVE_PATH, uni_df)

In [9]:
uni_df

ticker   open    high     low  close  openadj  highadj  \
permaticker date                                                                
101501      2012-01-03  BBUCQ   0.03   0.030   0.030   0.03     0.03    0.030   
            2012-01-04  BBUCQ   0.03   0.030   0.030   0.03     0.03    0.030   
            2012-01-05  BBUCQ   0.03   0.030   0.030   0.03     0.03    0.030   
            2012-01-06  BBUCQ   0.03   0.030   0.030   0.03     0.03    0.030   
            2012-01-09  BBUCQ   0.03   0.030   0.030   0.03     0.03    0.030   
...                       ...    ...     ...     ...    ...      ...      ...   
639045      2022-09-21   THRD  16.88  17.748  16.510  16.88    16.88   17.748   
            2022-09-22   THRD  16.88  18.360  16.350  16.75    16.88   18.360   
            2022-09-23   THRD  16.49  17.109  16.065  17.00    16.49   17.109   
639056      2022-09-22  GLSTU   9.97   9.990   9.960   9.98     9.97    9.990   
            2022-09-23  GLSTU   9.97   9.985   9.970   9.97     9.97    9.985   

                        lowadj  closeadj    volume  ...  alpha21  alpha22  \
permaticker date                                    ...                     
101501      2012-01-03   0.030      0.03       0.0  ...        1      NaN   
            2012-01-04   0.030      0.03       0.0  ...        1      NaN   
            2012-01-05   0.030      0.03       0.0  ...        1      NaN   
            2012-01-06   0.030      0.03       0.0  ...        1      NaN   
            2012-01-09   0.030      0.03       0.0  ...        1      NaN   
...                        ...       ...       ...  ...      ...      ...   
639045      2022-09-21  16.510     16.88  453144.0  ...        1      NaN   
            2022-09-22  16.350     16.75  266993.0  ...        1      NaN   
            2022-09-23  16.065     17.00  121250.0  ...        1      NaN   
639056      2022-09-22   9.960      9.98  606430.0  ...        1      NaN   
            2022-09-23   9.970      9.97  196845.0  ...        1      NaN   

                        alpha23  alpha24  alpha25  alpha26  alpha27  alpha28  \
permaticker date                                                               
101501      2012-01-03      0.0      NaN      NaN      NaN     -0.0      NaN   
            2012-01-04      0.0      NaN      NaN      NaN     -0.0      NaN   
            2012-01-05      0.0      NaN      NaN      NaN     -0.0      NaN   
            2012-01-06      0.0    -0.00      NaN      NaN     -0.0      NaN   
            2012-01-09      0.0    -0.00      NaN      NaN     -0.0      NaN   
...                         ...      ...      ...      ...      ...      ...   
639045      2022-09-21      0.0     1.12      NaN      NaN     -0.0      NaN   
            2022-09-22      0.0     0.25      NaN      NaN     -0.0      NaN   
            2022-09-23      0.0    -0.20      NaN      NaN     -0.0      NaN   
639056      2022-09-22      0.0      NaN      NaN      NaN     -0.0      NaN   
            2022-09-23      0.0      NaN      NaN      NaN     -0.0      NaN   

                        alpha29  alpha30  
permaticker date                          
101501      2012-01-03      NaN      NaN  
            2012-01-04      NaN      NaN  
            2012-01-05      NaN      NaN  
            2012-01-06      NaN      NaN  
            2012-01-09      NaN      NaN  
...                         ...      ...  
639045      2022-09-21      NaN      NaN  
            2022-09-22      NaN      NaN  
            2022-09-23      NaN      NaN  
639056      2022-09-22      NaN      NaN  
            2022-09-23      NaN      NaN  

[12835372 rows x 41 columns]